In [ ]:
!pip install requests-aws4auth

In [ ]:
!pip install -U opensearch-py==2.3.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 3.9 MB/s eta 0:00:00ta 0:00:01


In [ ]:
!pip install boto3


In [ ]:
!pip install -U retrying==1.3.4

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
import botocore
import time

In [ ]:
import json
import os
import boto3
import pprint
#from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss
import random
from retrying import retry
suffix = random.randrange(200, 900)

boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
bucket_name = "sagemaker-kb-910549846087" # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)

# Step 1 - Create OSS policies and collection and index 

In [ ]:
import boto3
import time
vector_store_name = "sagemaker-kb"
index_name = "sagemaker-readthedocs-io"
collection_name = "sagemaker-kb"
aoss_client = boto3_session.client('opensearchserverless')
bucket_name = 'sagemaker-kb-910549846087'
#client = boto3_session.client('opensearchserverless')
#service = 'aoss'
#region = 'us-west-2'
#bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
#bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']
bedrock_kb_execution_role_arn = "arn:aws:iam::910549846087:role/AmazonBedrockExecutionRoleForKnowledgeBase-sagemaker-kb"

In [ ]:
collection_list= aoss_client.list_collections(
    collectionFilters={
        'name': "sagemaker-kb",
        'status': 'ACTIVE'
    }
)

In [ ]:
print(collection_list['collectionSummaries'])

print(collection_list['collectionSummaries'][0]['id'])
#print(collection_list)

[{'arn': 'arn:aws:aoss:us-west-2:910549846087:collection/o42y2myczijzx7y8yvyl', 'id': 'o42y2myczijzx7y8yvyl', 'name': 'sagemaker-kb', 'status': 'ACTIVE'}]
o42y2myczijzx7y8yvyl


In [ ]:
collection_id = collection_list['collectionSummaries'][0]['id']
print(collection_id)

o42y2myczijzx7y8yvyl


In [ ]:
collection_id = collection_list['collectionSummaries'][0]['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

o42y2myczijzx7y8yvyl.us-west-2.aoss.amazonaws.com


In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = index_name
body_json = {
   "settings": {
      "index.knn": "true"
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# # It can take up to a minute for data access rules to be enforced
time.sleep(60)

In [ ]:
# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'sagemaker-readthedocs-io'}


# Step 2 - create knowledge base

In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": collection_list['collectionSummaries'][0]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = "sagemaker-docs"
description = "Amazon sagemaker letter knowledge base."
roleArn = bedrock_kb_execution_role_arn


In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [ ]:
pp.pprint(kb)

{ 'createdAt': datetime.datetime(2024, 1, 23, 9, 14, 49, 526873, tzinfo=tzlocal()),
  'description': 'Amazon sagemaker letter knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:910549846087:knowledge-base/VUPUHF4EJJ',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': 'VUPUHF4EJJ',
  'name': 'sagemaker-docs',
  'roleArn': 'arn:aws:iam::910549846087:role/AmazonBedrockExecutionRoleForKnowledgeBase-sagemaker-kb',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-west-2:910549846087:collection/o42y2myczijzx7y8yvyl',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                                 

In [ ]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [ ]:
# Create a DataSource in KnowledgeBase
data_source_name = "sagemaker-docs-s3"
create_ds_response = bedrock_agent_client.create_data_source(
    name = data_source_name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
pp.pprint(ds)

{ 'createdAt': datetime.datetime(2024, 1, 23, 9, 14, 52, 741945, tzinfo=tzlocal()),
  'dataSourceConfiguration': { 's3Configuration': { 'bucketArn': 'arn:aws:s3:::sagemaker-kb-910549846087'},
                               'type': 'S3'},
  'dataSourceId': 'DEZEANVLUZ',
  'description': 'Amazon sagemaker letter knowledge base.',
  'knowledgeBaseId': 'VUPUHF4EJJ',
  'name': 'sagemaker-docs-s3',
  'status': 'AVAILABLE',
  'updatedAt': datetime.datetime(2024, 1, 23, 9, 14, 52, 741945, tzinfo=tzlocal()),
  'vectorIngestionConfiguration': { 'chunkingConfiguration': { 'chunkingStrategy': 'FIXED_SIZE',
                                                               'fixedSizeChunkingConfiguration': { 'maxTokens': 512,
                                                                                                   'overlapPercentage': 20}}}}


In [ ]:
# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

{'ResponseMetadata': {'RequestId': '291d5612-7a0c-48ea-9052-4e12a0c0699b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:14:54 GMT',
   'content-type': 'application/json',
   'content-length': '547',
   'connection': 'keep-alive',
   'x-amzn-requestid': '291d5612-7a0c-48ea-9052-4e12a0c0699b',
   'x-amz-apigw-id': 'R_F-OGZ4vHcEKDA=',
   'x-amzn-trace-id': 'Root=1-65af838d-1a760eb21ac43b9f53b2fcd0'},
  'RetryAttempts': 0},
 'dataSource': {'knowledgeBaseId': 'VUPUHF4EJJ',
  'dataSourceId': 'DEZEANVLUZ',
  'name': 'sagemaker-docs-s3',
  'status': 'AVAILABLE',
  'description': 'Amazon sagemaker letter knowledge base.',
  'dataSourceConfiguration': {'type': 'S3',
   's3Configuration': {'bucketArn': 'arn:aws:s3:::sagemaker-kb-910549846087'}},
  'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE',
    'fixedSizeChunkingConfiguration': {'maxTokens': 512,
     'overlapPercentage': 20}}},
  'createdAt': datetime.datetime(2024, 1, 2

In [ ]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [ ]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

{ 'dataSourceId': 'DEZEANVLUZ',
  'ingestionJobId': 'SBYQRUZOCZ',
  'knowledgeBaseId': 'VUPUHF4EJJ',
  'startedAt': datetime.datetime(2024, 1, 23, 9, 14, 55, 524313, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 1, 23, 9, 14, 55, 524313, tzinfo=tzlocal())}


In [ ]:
# Get job 
while(job['status']!='COMPLETE' ):
  get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
  job = get_job_response["ingestionJob"]
pp.pprint(job)
time.sleep(40)

{ 'dataSourceId': 'DEZEANVLUZ',
  'ingestionJobId': 'SBYQRUZOCZ',
  'knowledgeBaseId': 'VUPUHF4EJJ',
  'startedAt': datetime.datetime(2024, 1, 23, 9, 14, 55, 524313, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 12,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 12},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2024, 1, 23, 9, 15, 19, 126518, tzinfo=tzlocal())}


In [ ]:
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'VUPUHF4EJJ'


In [ ]:
%store kb_id

Stored 'kb_id' (str)


## create agent 

In [ ]:
agent_name = "sagemaker-qa"
foundation_model = "anthropic.claude-v2"
instruction = "You are a Q&A agent that politely answers question from a knowledge."




In [ ]:
agent_response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    instruction=instruction,
    foundationModel=foundation_model,
    description='sagemaekr',
    idleSessionTTLInSeconds=123,
    agentResourceRoleArn="arn:aws:iam::910549846087:role/AmazonBedrockExecutionRoleForAgents_SageMakerQA",
    promptOverrideConfiguration={
        'promptConfigurations': [
            {
                'promptType': 'KNOWLEDGE_BASE_RESPONSE_GENERATION',
                'promptCreationMode': 'DEFAULT',
                #'promptState': 'ENABLED',
                #'basePromptTemplate': 'qa',
                # 'inferenceConfiguration': {
                #     'temperature': 0.5,
                #     'topK': 10,
                #     'maximumLength': 123,
                #     'stopSequences': [
                #         'string',
                #     ]
                # },
                'parserMode': 'DEFAULT'
            },
        ],
    }
)
time.sleep(7)

In [ ]:
print(agent_response)

{'ResponseMetadata': {'RequestId': 'aa01c406-9b8d-404a-9799-364fa1b8d8cb', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:16:05 GMT', 'content-type': 'application/json', 'content-length': '2281', 'connection': 'keep-alive', 'x-amzn-requestid': 'aa01c406-9b8d-404a-9799-364fa1b8d8cb', 'x-amz-apigw-id': 'R_GJXH7svHcEW7A=', 'x-amzn-trace-id': 'Root=1-65af83d5-61db0aa678f97ad279157d0b'}, 'RetryAttempts': 0}, 'agent': {'agentId': 'MR6TZVHNVO', 'agentName': 'sagemaker-qa', 'agentArn': 'arn:aws:bedrock:us-west-2:910549846087:agent/MR6TZVHNVO', 'instruction': 'You are a Q&A agent that politely answers question from a knowledge.', 'agentStatus': 'CREATING', 'foundationModel': 'anthropic.claude-v2', 'description': 'sagemaekr', 'idleSessionTTLInSeconds': 123, 'agentResourceRoleArn': 'arn:aws:iam::910549846087:role/AmazonBedrockExecutionRoleForAgents_SageMakerQA', 'createdAt': datetime.datetime(2024, 1, 23, 9, 16, 5, 350000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(202

In [ ]:
agent_id = agent_response["agent"]["agentId"]

In [ ]:
prepare_agent_response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(prepare_agent_response)

{'ResponseMetadata': {'RequestId': 'eb647f4d-326b-4846-bbad-c9c0f80dcf21', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:16:16 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'eb647f4d-326b-4846-bbad-c9c0f80dcf21', 'x-amz-apigw-id': 'R_GLIESSPHcEp7A=', 'x-amzn-trace-id': 'Root=1-65af83e0-7ef5306b6d4e301537654c60'}, 'RetryAttempts': 0}, 'agentId': 'MR6TZVHNVO', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 1, 23, 9, 16, 16, 617442, tzinfo=tzlocal())}


In [ ]:
agent_version = prepare_agent_response['agentVersion']
print(agent_version)

DRAFT


In [ ]:
knowledge_list_response = bedrock_agent_client.list_knowledge_bases(
    maxResults=123
)
print(knowledge_list_response)

{'ResponseMetadata': {'RequestId': '57d0c55e-0c5a-4a7f-b4cc-a00beeeced61', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:16:19 GMT', 'content-type': 'application/json', 'content-length': '204', 'connection': 'keep-alive', 'x-amzn-requestid': '57d0c55e-0c5a-4a7f-b4cc-a00beeeced61', 'x-amz-apigw-id': 'R_GLiGxDPHcEDlA=', 'x-amzn-trace-id': 'Root=1-65af83e3-243183a3522275c15359003c'}, 'RetryAttempts': 0}, 'knowledgeBaseSummaries': [{'knowledgeBaseId': 'VUPUHF4EJJ', 'name': 'sagemaker-docs', 'description': 'Amazon sagemaker letter knowledge base.', 'status': 'ACTIVE', 'updatedAt': datetime.datetime(2024, 1, 23, 9, 14, 49, 526873, tzinfo=tzlocal())}]}


In [ ]:
add_knowledge_base_response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion=agent_version,
    knowledgeBaseId=kb_id,
    description='Answer question about about Amazon sagemaker based only on information contained in knowledge base',
    knowledgeBaseState='ENABLED'
)
print(add_knowledge_base_response)
time.sleep(10)

{'ResponseMetadata': {'RequestId': 'a898d908-87f7-431d-9a6b-a358d435097d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:16:21 GMT', 'content-type': 'application/json', 'content-length': '291', 'connection': 'keep-alive', 'x-amzn-requestid': 'a898d908-87f7-431d-9a6b-a358d435097d', 'x-amz-apigw-id': 'R_GL3EtgPHcEG5w=', 'x-amzn-trace-id': 'Root=1-65af83e5-505169ec3bd287060a7736a2'}, 'RetryAttempts': 0}, 'agentKnowledgeBase': {'knowledgeBaseId': 'VUPUHF4EJJ', 'description': 'Answer question about about Amazon sagemaker based only on information contained in knowledge base', 'createdAt': datetime.datetime(2024, 1, 23, 9, 16, 21, 291414, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 1, 23, 9, 16, 21, 291414, tzinfo=tzlocal()), 'knowledgeBaseState': 'ENABLED'}}


In [ ]:
prepare_agent_response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(prepare_agent_response)
time.sleep(10)

{'ResponseMetadata': {'RequestId': '8556c7ee-9c80-4404-ad8b-c5efc77b5454', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:16:49 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '8556c7ee-9c80-4404-ad8b-c5efc77b5454', 'x-amz-apigw-id': 'R_GQPH1YvHcEXNA=', 'x-amzn-trace-id': 'Root=1-65af8401-6bfe7ad83ad6240421b1e08d'}, 'RetryAttempts': 0}, 'agentId': 'MR6TZVHNVO', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 1, 23, 9, 16, 49, 388795, tzinfo=tzlocal())}


In [ ]:
agent_alias_name = "Sagemaker-bedrock"
agent_alias_response = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name,
    description="testing the agent via knowledge"
)
print(agent_alias_response)

{'ResponseMetadata': {'RequestId': '644c1fdf-5603-4ff2-bcb4-29ebfd40ef2f', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 23 Jan 2024 09:17:09 GMT', 'content-type': 'application/json', 'content-length': '389', 'connection': 'keep-alive', 'x-amzn-requestid': '644c1fdf-5603-4ff2-bcb4-29ebfd40ef2f', 'x-amz-apigw-id': 'R_GTYFD0vHcEPfQ=', 'x-amzn-trace-id': 'Root=1-65af8415-747ec8f934f1b1a00f8ac213'}, 'RetryAttempts': 0}, 'agentAlias': {'agentId': 'MR6TZVHNVO', 'agentAliasId': '5AAANZVPEB', 'agentAliasName': 'Sagemaker-bedrock', 'agentAliasArn': 'arn:aws:bedrock:us-west-2:910549846087:agent-alias/MR6TZVHNVO/5AAANZVPEB', 'description': 'testing the agent via knowledge', 'routingConfiguration': [], 'createdAt': datetime.datetime(2024, 1, 23, 9, 17, 9, 451520, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 1, 23, 9, 17, 9, 451520, tzinfo=tzlocal()), 'agentAliasStatus': 'CREATING'}}


In [ ]:
print(agent_alias_response['agentAlias']['agentAliasId'])
agent_alias_id = agent_alias_response['agentAlias']['agentAliasId']

5AAANZVPEB


In [ ]:
#bedrock_agent_invoke_client = boto3_session.client('bedrock', region_name=region_name)
bedrock_agent_invoke_client = boto3_session.client('bedrock-agent-runtime',region_name=region_name)

In [ ]:
print(agent_id,agent_alias_id,)

MR6TZVHNVO 5AAANZVPEB


In [ ]:
# import uuid
# session_id = uuid.uuid4().hex

# def invoke_agent(prompt):
#     """
#     Sends a prompt for the agent to process and respond to.

#     :param agent_id: The unique identifier of the agent to use.
#     :param agent_alias_id: The alias of the agent to use.
#     :param session_id: The unique identifier of the session. Use the same value across requests
#                        to continue the same conversation.
#     :param prompt: The prompt that you want Claude to complete.
#     :return: Inference response from the model.
#     """

    
#     response = bedrock_agent_invoke_client.invoke_agent(
#         agentId=agent_id,
#         agentAliasId=agent_alias_id,
#         sessionId=session_id,
#         inputText=prompt,
#     )

#     completion = ""

#     for event in response.get("completion"):
#         chunk = event["chunk"]
#         completion = completion + chunk["bytes"].decode()

#     return completion

# def chat_with_agent(agent_alias):
#     print("-" * 88)
#     print("The agent is ready to chat.")
#     print("Try asking for the date or time. Type 'exit' to quit.")

#     # Create a unique session ID for the conversation
#     session_id = uuid.uuid4().hex
#     print(session_id)
    

# #     while True:
# #         prompt = q.ask("Prompt: ", q.non_empty)

# #         if prompt == "exit":
# #             break


#     response = invoke_agent("what is sagemaker")

#     print(f"Agent: {response}")
        

# chat_with_agent(agent_alias_id)

----------------------------------------------------------------------------------------
The agent is ready to chat.
Try asking for the date or time. Type 'exit' to quit.
716a7731a96446c9be67987a1594e70a
Agent: SageMaker is Amazon's managed service for building, training, and deploying machine learning models.


## delete agent, alias and knowledge base

In [ ]:
# print("Deleting agent alias...")
# delete_agent_alias_response = bedrock_agent_client.delete_agent_alias(
#     agentId=agent_id,
#     agentAliasId=agent_alias_id
# )

# print("agent alias deleted")
# print("--------------------------------------")
# print("Deleting agent...")
# agent_status = bedrock_agent_client.delete_agent(agentId=agent_id)

# print("------------------------------------------")
# print("Deleting knowledge base")
# delete_kb_response = bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
# print("knowledge base deleted")

Deleting agent alias...
agent alias deleted
--------------------------------------
Deleting agent...
------------------------------------------
Deleting knowledge base
knowledge base deleted
